In [169]:
# 형태소 분석 후 txt 로 저장
make_tokenized_text = 0 
if make_tokenized_text:
    import time
    s = time.time()
    from konlpy.tag import Twitter
    twitter = Twitter()

    rawtext='merged_comments.txt'
    outputtext = 'merged_comments_tokenized.txt'
    with open(rawtext,encoding='utf8') as f:
        with open(outputtext, 'w', encoding='utf8') as output:
            line = f.readline()
            while line:
                line=line.split('\t')
                sentence = line[1]
                score = line[2]
                twit = twitter.pos(sentence, stem=True, norm=True)
    #             sentence = [word + "/" + pos[0] + pos[-1] for (word, pos) in twit if pos != 'Josa']
                sentence = [word for (word, pos) in twit if pos != 'Josa']
                outputline = " ".join(sentence) + "\t" + score
    #             print(outputline)
                output.writelines(outputline)
                line=f.readline()
    e = time.time()
    print( (e-s)/60 )

4.958832025527954


In [1]:
#데이터 체크 
rawtext= 'merged_comments.txt'
with open(rawtext, encoding= 'utf8')as f:
    for _ in range(10):
        print(f.readline().strip())


45290	크리스토퍼 놀란 에게 우리는 놀란 다	10
45290	인셉션 정말 흥미진진하게 봤었고 크리스토퍼 놀란 감독님 신작 인터스텔라도 이번주 일요일에 보러갑니다 완전 기대중	10
45290	놀란이면 무조건 봐야 된다 왜냐하면 모든 작품을 다 히트 쳤으니깐	10
45290	나는 감탄할 준비가 되어있다	10
45290	얘들아 오늘나오는거지 밤에 ㅋㅋ 오늘 보러가야겟다	10
45290	이제 죽어도 여한이 없다	10
45290	기대감에 잠도 안온다	10
45290	나랑 같이 봐 줄까 ㅎ	10
45290	우선 명감독들이 모였고 미국에서도 극찬을 받았더군	10
45290	드디어 내일 꼭본다	10


In [3]:
def load_reviews(fname, n_limit=-1): 
    with open(fname, encoding='utf-8') as f:
        idxs = []
        scores = []
        texts = []
        
        for i, doc in enumerate(f):
            if n_limit > 0 and i >= n_limit:
                break
                
            try:
                idx, text, score = doc.strip().split('\t')
                idxs.append(idx)
                texts.append(text.strip())
                scores.append(int(score))
                
            except Exception as e:
                print(e)
                print(doc)
                continue
                
    return idx, texts, scores

idx, texts, scores = load_reviews(rawtext)
print(len(scores))

294493


In [4]:
from collections import Counter
word_count = Counter( [word for text in texts for word in text.strip().split()] )

# 빈도수 상위 20개 단어 출력
from pprint import pprint
top = sorted(word_count.items(), key=lambda x: x[1], reverse =True)[:20]
pprint(top)



# n 번 이상 출현한 단어들
for limit in [1,2,3,4,5,6,7]:
    word_count_limit={ word for word,frequency in word_count.items()
                     if frequency >=limit}
    print('limit = %d, the number of words : %d'%(limit, len(word_count_limit)))




[('영화', 40000),
 ('정말', 18266),
 ('진짜', 14207),
 ('너무', 13397),
 ('이', 7893),
 ('영화를', 7006),
 ('그냥', 6844),
 ('더', 6560),
 ('최고의', 6394),
 ('보고', 5899),
 ('좀', 5725),
 ('수', 5671),
 ('영화가', 5588),
 ('최고', 5347),
 ('영화는', 5308),
 ('잘', 5039),
 ('꼭', 4944),
 ('ㅋㅋ', 4865),
 ('본', 4655),
 ('다', 4609)]
limit = 1, the number of words : 400649
limit = 2, the number of words : 100035
limit = 3, the number of words : 61982
limit = 4, the number of words : 46182
limit = 5, the number of words : 37222
limit = 6, the number of words : 31295
limit = 7, the number of words : 27206


In [6]:
# stopwords
possitive_limit=9
negative_limit=3

possitive_word_count = Counter( [word for (text,score) in zip(texts,scores) for word in text.strip().split() if score >= possitive_limit ]  )
negative_word_count = Counter( [word for (text,score) in zip(texts,scores) for word in text.strip().split() if score <= negative_limit ]  )

top_possitive = [word for (word,count) in (sorted(possitive_word_count.items(), key =lambda x:x[1], reverse= True)[:50]) ]
top_negative=[word for (word,count) in sorted(negative_word_count.items(), key = lambda x:x[1], reverse = True)[:50]]

stopwords = list(set(top_possitive) & set(top_negative))
pprint(stopwords)

['영화가',
 '이렇게',
 '이',
 '그',
 '본',
 '영화',
 '이런',
 '영화는',
 '영화를',
 '봤는데',
 '정말',
 '완전',
 '왜',
 '평점',
 '내',
 '보고',
 '난',
 '다',
 '내가',
 '너무',
 '이건',
 'ㅋㅋ',
 '더',
 '좀',
 '그냥',
 '진짜',
 'ㅋ']


In [7]:
# 빈도 10 이상인 단어들만 남기기 + stopwords 제거

word_count_limit = { word for word, frequency in word_count.items()
                   if frequency >=10 and word not in stopwords}


def my_word_tokenizer(sentence_in_string):
    return [word for word in sentence_in_string.strip().split()
            if word in word_count_limit]

print(len(word_count_limit))
print(texts[2])
print(my_word_tokenizer(texts[2]))


19737
놀란이면 무조건 봐야 된다 왜냐하면 모든 작품을 다 히트 쳤으니깐
['무조건', '봐야', '된다', '왜냐하면', '모든', '작품을']


In [8]:
#영화 평점 별 리뷰 수
from collections import Counter
for score, score_freq in sorted(Counter(scores).items()):
    print('score = %d: (%d, %.3f perc)' % (score, score_freq, 100*score_freq/len(scores)))

score = 1: (56122, 19.057 perc)
score = 2: (4725, 1.604 perc)
score = 3: (4547, 1.544 perc)
score = 4: (4062, 1.379 perc)
score = 5: (7697, 2.614 perc)
score = 6: (7588, 2.577 perc)
score = 7: (11338, 3.850 perc)
score = 8: (20311, 6.897 perc)
score = 9: (25528, 8.668 perc)
score = 10: (152575, 51.809 perc)


In [9]:
# 1~3 점은 -1
# 9,10점은 1
# 나머지는 사용하지 않음

train_texts = []
train_label = []


for review, score in zip(texts, scores):
    if negative_limit <  score <  possitive_limit:
        continue
    tokenized_review = my_word_tokenizer(review)
#     words = [word for word in review.split() if word in word_dictionary]
    if not tokenized_review:
        continue
  
    train_texts.append(tokenized_review)
    train_label.append(1 if score >= possitive_limit else -1)

print('train data: %d --> %d' % (len(texts), len(train_texts)))
for label, label_freq in Counter(train_label).items():
    print('label = %d: (%d, %.3f perc)' % (label, label_freq, 100*label_freq/len(train_label)))

train data: 294493 --> 221252
label = 1: (162846, 73.602 perc)
label = -1: (58406, 26.398 perc)


In [10]:
# 단어의 빈도수로 벡터화

from sklearn.feature_extraction.text import CountVectorizer

#train_texts가 string 이라면 tokenizer 설정 필요 x
vectorizer= CountVectorizer(
    tokenizer = lambda x:x , lowercase = False, 
  
    )

train_x= vectorizer.fit_transform(train_texts)
print(type(train_x) ) 
train_x



<class 'scipy.sparse.csr.csr_matrix'>


<221252x19735 sparse matrix of type '<class 'numpy.int64'>'
	with 1040849 stored elements in Compressed Sparse Row format>

In [12]:
# 특정 단어에 해당하는 인덱스 알아보기

vocab2int = vectorizer.vocabulary_ #dictionary, {word: index} 형태
print(vocab2int['재미'])

int2vocab = [word for word,index in sorted # 단어를 철자순으로 정렬한 list
             (vocab2int.items(), key=lambda x:x[1])] 


print(int2vocab[15378])


15378
재미


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
logistic = LogisticRegression()
logistic.fit(train_x, train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
# n 번째 리뷰를 골라 predict 해보기

for idx in [14,100,1000,199999,200006]:
    print('text: %s\n' % (train_texts[idx]))
    
    prob = logistic.predict_proba(train_x[idx])[0]
    pred_class = logistic.predict(train_x[idx])
    actual_class = train_label[idx]
    
    if actual_class ==1:
        actual_class="positive"
    else:
        actual_class="negative"
    
    print('predicted class prob: (negative= %.3f, positive= %.3f' 
          % tuple(prob))
    print('predicted class = %s' %('positive') if pred_class == 1 
          else 'predicted class = %s'%('negative'))
    print('actual class = %s'%(actual_class))
    print('\n%s\n' % ('-'*50))

    
    

text: ['재미없다', '이상', '10자']

predicted class prob: (negative= 0.740, positive= 0.260
predicted class = negative
actual class = negative

--------------------------------------------------

text: ['대박나세요']

predicted class prob: (negative= 0.123, positive= 0.877
predicted class = positive
actual class = positive

--------------------------------------------------

text: ['쿠퍼가', '장면이', '잊혀지질', '않네요']

predicted class prob: (negative= 0.124, positive= 0.876
predicted class = positive
actual class = positive

--------------------------------------------------

text: ['감독', '이름', '조심해라']

predicted class prob: (negative= 0.859, positive= 0.141
predicted class = negative
actual class = negative

--------------------------------------------------

text: ['나', '혼자만', 'ㄷㄷ']

predicted class prob: (negative= 0.345, positive= 0.655
predicted class = positive
actual class = positive

--------------------------------------------------



In [16]:
# 사용자로부터 새로운 인풋을 받아서 predict 하고 싶을 때

# vectorizer_ = vectorizer.fit(train_texts)
# new_input = vectorizer_.transform( ["와 정말 재미있는 영화네요 이건 긍정적인 문장일까요?".split()] )
# new_pred_class = logistic.predict(new_input)
# if new_pred_class == 1:
#     print("positive")
# else:
#     print("negative")


positive


In [21]:
#coefficients 살펴보기

# 긍정리뷰로 예측하는데 영향을 미친 단어들 상위 10
coefficients = logistic.coef_.tolist()
sorted_coefficients = sorted(enumerate(coefficients[0]), 
                             key=lambda x:x[1], reverse=True)

for index, coef in sorted_coefficients[:10]:
    print('%s (%.3f)' % (int2vocab[index], coef))

알이즈웰 (4.322)
최고입니다 (3.525)
기대됩니다 (3.236)
관람객재밌어요 (3.219)
10점준다 (3.160)
좋았어요 (3.039)
웰 (3.022)
재미있었습니다 (3.016)
꿀잼 (2.962)
인생영화 (2.928)


In [22]:
# 부정리뷰로 예측하는데 영향을 미친 단어들 상위 10
# coefficients = logistic.coef_.tolist()
sorted_coefficients = sorted(enumerate(coefficients[0]), 
                             key=lambda x:x[1])


for index, coef in sorted_coefficients[:10]:

    print('%s (%.3f)' % (int2vocab[index], coef))

0점은 (-4.377)
최악 (-4.216)
최악의 (-4.191)
0점이 (-3.974)
1점준다 (-3.837)
돈아깝다 (-3.749)
쓰레기영화 (-3.662)
노잼 (-3.570)
1점대 (-3.544)
최악의영화 (-3.491)


In [28]:
# 5 - FOLD CORSSVAL 

y_predicted = cross_val_predict(logistic,train_x,train_label,cv=5)
conf_mat = confusion_matrix(train_label,y_predicted, labels=[1, -1])
TP, FN, FP, TN =conf_mat[0][0], conf_mat[0][1], conf_mat[1][0], conf_mat[1][1]

print(conf_mat)
print(" accuacy: %.3f\n recall : %.3f\n precision: %.3f"%( (TP+TN)/(TP+TN+FN+FP), TP/(TP+FN), TP/(TP+FP)  ) )



[[150112  12734]
 [ 20842  37564]]
 accuacy: 0.848
 recall : 0.922
 precision: 0.878
